# 개고양이 분류 모델 구축 예제

## 데이터셋 생성

In [1]:

import os
import numpy as np
import cv2

images = []
labels = []
for image_file_name in os.listdir('dog_cat_images'):
    if image_file_name == '.ipynb_checkpoints': # ignore .ipynb_checkpoints 
        continue 
    image_path = os.path.join('dog_cat_images', image_file_name)
    #print(f'image_path: {image_path}')
    img = cv2.imread(image_path)
    img = cv2.resize(img, (128, 128))
    images.append(img)
    if image_file_name[:3] == 'dog':
        #print(f'label: {0}')
        labels.append(0)
    else:
        #print(f'label: {1}')
        labels.append(1)
images = np.array(images)
labels = np.array(labels)
print(f'images.shape: {images.shape}')
print(f'labels.shape: {labels.shape}')

images.shape: (400, 128, 128, 3)
labels.shape: (400,)


## train-test-split

In [2]:
from sklearn.model_selection import train_test_split

# train-test
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, shuffle=True)
# validation-test
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, shuffle=True)

## CNN Model

In [5]:
# input => (128,128,3)

# conv2D
# maxpool2D
# Conv2D
# MaxPool2D
# Conv2D
# MaxPool2D
# Flatten
# Dense
# Dense
# Dense => node 1개, Sigmoid

# Loss Func → Binary Crossentropy

import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 63, 63, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 14, 14, 128)      

In [6]:
model.fit(X_train, y_train, epochs = 10, batch_size = 32)

Epoch 1/10
10/10 [==============================] - 2s 157ms/step - loss: 12.2506 - accuracy: 0.5250
Epoch 2/10
10/10 [==============================] - 2s 158ms/step - loss: 0.6728 - accuracy: 0.5813
Epoch 3/10
10/10 [==============================] - 2s 158ms/step - loss: 0.4535 - accuracy: 0.8031
Epoch 4/10
10/10 [==============================] - 2s 155ms/step - loss: 0.2755 - accuracy: 0.9344
Epoch 5/10
10/10 [==============================] - 2s 157ms/step - loss: 0.1812 - accuracy: 0.9438
Epoch 6/10
10/10 [==============================] - 2s 160ms/step - loss: 0.1269 - accuracy: 0.9594
Epoch 7/10
10/10 [==============================] - 2s 159ms/step - loss: 0.0728 - accuracy: 0.9969
Epoch 8/10
10/10 [==============================] - 2s 163ms/step - loss: 0.0391 - accuracy: 0.9969
Epoch 9/10
10/10 [==============================] - 2s 169ms/step - loss: 0.0274 - accuracy: 0.9969
Epoch 10/10
10/10 [==============================] - 2s 165ms/step - loss: 0.0120 - accuracy: 1.000

In [7]:
# 모델 테스트
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)

print(f'\nTest accuracy: {test_acc}, Test loss: {test_loss}')

3/3 - 0s - loss: 1.7171 - accuracy: 0.6375 - 331ms/epoch - 110ms/step

Test accuracy: 0.637499988079071, Test loss: 1.7170813083648682


# Hyperparams Tuning - Bayesian Opt !

In [4]:
import tempfile
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras import Sequential
from kerastuner.tuners import BayesianOptimization

def build_model(hp):
    model = Sequential([
        Conv2D(hp.Int('conv_1_filters', min_value=32, max_value=128, step=32), (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(hp.Int('conv_2_filters', min_value=64, max_value=256, step=32), (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(hp.Int('conv_3_filters', min_value=128, max_value=512, step=32), (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(hp.Int('dense_1_units', min_value=32, max_value=128, step=32), activation='relu'),
        Dense(hp.Int('dense_2_units', min_value=16, max_value=64, step=16), activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

with tempfile.TemporaryDirectory() as tmp_dir:
    tuner = BayesianOptimization(
        build_model,
        objective='val_accuracy',
        max_trials=10,
        executions_per_trial=1,
        directory=tmp_dir,  # Use the temporary directory
        project_name='bayesian_optimization'
    )

tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

model = build_model(best_hps)
model.summary()


Trial 10 Complete [00h 00m 46s]
val_accuracy: 0.6000000238418579

Best val_accuracy So Far: 0.7250000238418579
Total elapsed time: 00h 15m 14s
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 126, 126, 128)     3584      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 63, 63, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 61, 61, 224)       258272    
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 30, 30, 224)       0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 28, 28,

In [17]:
# Assuming the tuner has completed its search
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Print the best number of units in the dense layers and the best number of filters in the Conv2D layers
print(f"""
Best hyperparameters found: 
- First Conv2D filters: {best_hps.get('conv_1_filters')}
- Second Conv2D filters: {best_hps.get('conv_2_filters')}
- Third Conv2D filters: {best_hps.get('conv_3_filters')}
- First Dense layer units: {best_hps.get('dense_1_units')}
- Second Dense layer units: {best_hps.get('dense_2_units')}
""")


Best hyperparameters found: 
- First Conv2D filters: 96
- Second Conv2D filters: 160
- Third Conv2D filters: 256
- First Dense layer units: 32
- Second Dense layer units: 32

